### Advent Of Code Day 15: Chiton

In [78]:
import numpy as np

riskLevel = []

with open("./input", 'r') as file:
    for line in file:
        riskLevel.append([int(risk) for risk in line[:-1]])

riskLevel = np.array(riskLevel)
riskLevel.shape

(100, 100)

In [79]:
# I can use Dijkstra or UCS for searching the best path among all the possible solutions

import util

def uniformCostSearch(riskLevel, terminalState):
    """Search the node of least total cost first."""

    table = dict() # States lookup table for rebuilding the path
    container = util.PriorityQueue() # Priority FIFO container as frontier

    # Node encoding: (state, parent, action, accumulated cost) 
    # Zero start state priority as it cost nothing to get there
    start_node = ((0, 0), None, None, 0)
    container.push(start_node, 0) 

    while not container.isEmpty():
        node = container.pop()
        state, parent, action, cost = node

        # If the state hasn't been visited or it has but with greater cost,
        # updates its table entry with its new node relevant information.
        if state not in table or cost < table[state][2]:
            table[state] = (parent, action, cost)

            # Ends searching if its a goal state
            if state == terminalState: break

            legalActions = []
            if state[0] < terminalState[0]: legalActions.append((1,0))
            if state[1] < terminalState[1]: legalActions.append((0,1))
            if state[0] > 0: legalActions.append((-1, 0))
            if state[1] > 0: legalActions.append((0, -1))
            
            # For each of its successors, create a node and push it to the container
            for next_action in legalActions:
                next_state = (state[0] + next_action[0], state[1] + next_action[1])
                next_cost = cost + riskLevel[next_state]
                
                # UCS's priority: g(n) (accumulated cost from start node)
                priority = next_cost
                container.push((next_state, state, next_action, next_cost), priority)

    return getPathSolution(state, table), cost

def getPathSolution(state, table):
    """ Gets the action path solution, beggining from the last visited state. """

    path = []
    if state not in table: return path

    # Reconstruct the path by going backwards throught the lookup table,
    # while there's still actions to append.
    previous_state, action, cost = table[state]

    while action:
        path.append(previous_state)
        previous_state, action, cost = table[previous_state]
    
    return path[::-1]


In [80]:
bestPath, pathCost = uniformCostSearch(riskLevel, (99, 99))
pathCost

472

**Part Two**

The only difference is the larger risk map. 

In [81]:
# New Risk Level for each tile repeated ro the right or down
def newRiskLevel(riskLevel, i, j): return (riskLevel + i + j - 1) % 9 + 1

# Build your larger risk map using the numpy's block function
newRiskMap = np.block([[newRiskLevel(riskLevel, i, j) for j in range(5)] for i in range(5)])
newRiskMap.shape

(500, 500)

In [82]:
# Rock and Roll!
bestPath, pathCost = uniformCostSearch(newRiskMap, (499, 499))
pathCost

2851